---



# <font color='#ECA702'>**Análisis Multimodal de Síntomas en la Enfermedad de Parkinson**</font>

In [ ]:
#@title **Importar librerías**

# Manipulación de datos
import os
import cv2
import numpy as np
import librosa

# Pytorch essentials
import torch

# Utils
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
#@title **Cargamos los datos desde Drive**

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/data_parkinson')
print(os.getcwd())

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1XRruCPRodR4OhRCRZYKsyODqa_GuBkfC/data_parkinson


# <font color='#4C5FDA'> **Reducción de la dimensionalidad**</font> 

<font color='#52F17F'>**PCA (Principal Component Analysis)**</font>: PCA es una técnica de reducción de dimensionalidad que transforma las características de los frames en un nuevo espacio de características, reduciendo la dimensión mientras conserva la información más significativa.

En un problema multiclase haría algo así:

<center>
  <img src="https://www.askpython.com/wp-content/uploads/2020/10/Visualizing-digits-data-in-2-dimensions-2-1024x512.jpeg.webp" width="600" height="400">
</center>

En el cual las clases iniciales eran las siguientes:

<center>
  <img src="https://machinelearningmastery.com/wp-content/uploads/2019/02/Plot-of-a-Subset-of-Images-from-the-MNIST-Dataset-1024x768.png" width="600" height="400">
</center>

https://www.kaggle.com/code/tilii7/dimensionality-reduction-pca-tsne - PCA y tSNE

https://www.askpython.com/python/examples/principal-component-analysis-for-image-data - PCA

https://www.kaggle.com/code/arthurtok/interactive-intro-to-dimensionality-reduction - PCA y tSNE